# Experiment Analysis Framework

This notebook aggregates prior training artifacts from **neural-network-lab-python**, surfaces diagnostic visualizations, and recommends data-driven hyperparameter refinements for future experiments. It is designed to be reusable across training runs with minimal manual setup.

## Workflow Overview

1. Validate the presence of required configs, logs, scalers, and weight checkpoints.
2. Load active and historical configuration payloads and align them with training outcomes.
3. Ingest `loss_history.csv`, `training_results.csv`, and particle simulation data for analytics.
4. Reconstruct the latest model checkpoint, generate predictions, and evaluate residuals.
5. Render visual diagnostics (loss curves, learning-rate sweeps, residual histograms, correlation heatmap).
6. Summarize run health, recommend hyperparameter sweeps, and capture actionable next steps.

In [7]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Sequence, Tuple

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from IPython.display import Markdown, display

from advanced_neural_network import AdvancedNeuralNetwork
from data_processing import complete_data_pipeline, load_and_validate_data
from ml_utils import compute_loss_weights
from weight_constraints import BinaryWeightConstraintChanges, BinaryWeightConstraintMax, OscillationDampener

pd.options.display.max_rows = 60
pd.options.display.float_format = '{:,.4f}'.format

sns.set_theme(style="whitegrid")

In [8]:
PROJECT_NAME = "neural-network-lab-python"

INPUT_FEATURES = [
    "mass",
    "initial_velocity_x",
    "initial_velocity_y",
    "initial_position_x",
    "initial_position_y",
    "charge",
    "magnetic_field_strength",
    "simulation_time"
]

OUTPUT_TARGETS = [
    "final_velocity_x",
    "final_velocity_y",
    "final_position_x",
    "final_position_y",
    "kinetic_energy",
    "trajectory_length"
]

ANALYSIS_SEED = 42

np.random.seed(ANALYSIS_SEED)
tf.random.set_seed(ANALYSIS_SEED)

def resolve_project_paths() -> Dict[str, Path]:
    """Resolve key project directories relative to this notebook."""
    root = Path.cwd()

    if root.name != PROJECT_NAME:
        for parent in root.parents:
            if parent.name == PROJECT_NAME: root = parent

    config_dir = root / "ml_config"

    output_dir = root / "training_output"

    analysis_dir = output_dir / "analysis"

    figures_dir = analysis_dir / "figures"

    analysis_dir.mkdir(parents=True, exist_ok=True)

    figures_dir.mkdir(parents=True, exist_ok=True)

    return {
        "project_root": root,
        "config_dir": config_dir,
        "output_dir": output_dir,
        "analysis_dir": analysis_dir,
        "figures_dir": figures_dir,
        "data_path": root / "particle_data.csv",
        "scaler_X": root / "scaler_X.pkl",
        "scaler_y": root / "scaler_y.pkl"
    }

def validate_required_artifacts(paths: Dict[str, Path]) -> pd.DataFrame:
    """Check presence and metadata of required artifacts."""
    required = {
        "model_config": paths["config_dir"] / "model_config.json",
        "training_config": paths["config_dir"] / "training_config.json",
        "loss_history": paths["output_dir"] / "loss_history.csv",
        "training_results": paths["output_dir"] / "training_results.csv",
        "configuration_log": paths["output_dir"] / "configuration_log.csv",
        "particle_data": paths["data_path"],
        "scaler_X": paths["scaler_X"],
        "scaler_y": paths["scaler_y"]
    }

    records: List[Dict[str, Any]] = []

    for label, path in required.items():
        exists = path.exists()

        size = path.stat().st_size if exists else None

        records.append({
            "artifact": label,
            "path": str(path),
            "exists": exists,
            "size_bytes": size
        })

    status_df = pd.DataFrame(records)

    return status_df

def list_checkpoint_weights(paths: Dict[str, Path]) -> pd.DataFrame:
    """List available weight checkpoints with epoch metadata."""
    pattern = "model_weights_epoch_*.weights.h5"

    checkpoint_files = sorted(paths["project_root"].glob(pattern))

    rows: List[Dict[str, Any]] = []

    for file_path in checkpoint_files:
        name = file_path.name

        parts = name.split("_")

        epoch_token = parts[3] if len(parts) > 3 else parts[-1]

        epoch = int(epoch_token.replace(".weights.h5", "")) if epoch_token else None

        rows.append({
            "epoch": epoch,
            "name": name,
            "path": str(file_path),
            "modified": pd.Timestamp(file_path.stat().st_mtime, unit="s"),
            "size_bytes": file_path.stat().st_size
        })

    checkpoint_df = pd.DataFrame(rows)

    if checkpoint_df.empty: return checkpoint_df

    checkpoint_df = checkpoint_df.sort_values("epoch").reset_index(drop=True)

    return checkpoint_df

In [9]:
def load_configs(paths: Dict[str, Path]) -> Tuple[Dict[str, Any], Dict[str, Any], pd.DataFrame]:
    """Load active configs and historical configuration snapshots."""
    model_config_path = paths["config_dir"] / "model_config.json"

    training_config_path = paths["config_dir"] / "training_config.json"

    with model_config_path.open() as handle:
        model_config = json.load(handle)

    with training_config_path.open() as handle:
        training_config = json.load(handle)

    snapshots: List[Dict[str, Any]] = []

    for config_path in sorted(paths["output_dir"].glob("training_config_*.json")):
        with config_path.open() as handle:
            payload = json.load(handle)

        combined: Dict[str, Any] = {
            "config_id": payload.get("config_id"),
            "timestamp": payload.get("timestamp")
        }

        model_payload = payload.get("model_config", {})

        for key, value in model_payload.items():
            combined[key] = value

        training_payload = payload.get("training_config", {})

        for key, value in training_payload.items():
            combined[f"train_{key}"] = value

        summary_payload = payload.get("performance_summary", {})

        combined["best_r2"] = summary_payload.get("best_r2")
        combined["final_r2"] = summary_payload.get("current_r2")
        combined["best_epoch"] = summary_payload.get("best_r2_epoch")
        combined["avg_epoch_time"] = summary_payload.get("avg_epoch_time")
        combined["total_training_time"] = summary_payload.get("total_training_time")
        combined["weight_modifications_used"] = summary_payload.get("weight_modifications_used")

        snapshots.append(combined)

    snapshots_df = pd.DataFrame(snapshots)

    if not snapshots_df.empty:
        snapshots_df["timestamp"] = pd.to_datetime(snapshots_df["timestamp"])

    return model_config, training_config, snapshots_df

def load_training_logs(paths: Dict[str, Path]) -> Dict[str, pd.DataFrame]:
    """Load loss history and training results with derived analytics."""
    loss_path = paths["output_dir"] / "loss_history.csv"

    results_path = paths["output_dir"] / "training_results.csv"

    loss_records = pd.read_csv(loss_path)

    epoch_summary = (
        loss_records.groupby("epoch").agg(
            combined_loss_mean=("combined_loss", "mean"),
            combined_loss_std=("combined_loss", "std"),
            mae_mean=("mae", "mean"),
            mse_mean=("mse", "mean")
        ).reset_index()
    )

    results_df = pd.read_csv(results_path)

    results_df["timestamp"] = pd.to_datetime(results_df["timestamp"])

    results_df["epoch"] = results_df["epoch"].astype(int)

    results_df["val_loss_delta"] = results_df["val_loss"].diff()

    results_df["train_val_gap"] = results_df["val_loss"] - results_df["train_loss"]

    merged_metrics = results_df.merge(epoch_summary, on="epoch", how="left")

    merged_metrics["val_loss_rolling"] = merged_metrics["val_loss"].rolling(5, min_periods=1).mean()

    merged_metrics["train_loss_rolling"] = merged_metrics["train_loss"].rolling(5, min_periods=1).mean()

    analytics = {
        "loss_records": loss_records,
        "epoch_summary": epoch_summary,
        "results": results_df,
        "merged_metrics": merged_metrics
    }

    return analytics

def load_scalers(paths: Dict[str, Path]) -> Tuple[Any, Any]:
    """Load cached scalers, regenerating them via training pipeline if missing."""
    scaler_X_path = paths["scaler_X"]

    scaler_y_path = paths["scaler_y"]

    try:
        scaler_X = joblib.load(scaler_X_path)
    except FileNotFoundError:
        complete_data_pipeline(csv_path=str(paths["data_path"]))

        scaler_X = joblib.load(scaler_X_path)

    try:
        scaler_y = joblib.load(scaler_y_path)
    except FileNotFoundError:
        complete_data_pipeline(csv_path=str(paths["data_path"]))

        scaler_y = joblib.load(scaler_y_path)

    return scaler_X, scaler_y

def load_particle_data(paths: Dict[str, Path]) -> pd.DataFrame:
    """Load particle simulation data with validation safeguards."""
    dataset = load_and_validate_data(csv_path=str(paths["data_path"]))

    return dataset

In [10]:
def build_model_from_config(model_config: Dict[str, Any], training_config: Dict[str, Any]) -> tf.keras.Model:
    """Instantiate a compiled model that mirrors the training setup."""
    config_payload = dict(model_config)

    config_payload.update(training_config)

    config_payload.setdefault("enable_weight_oscillation_dampener", True)

    input_shape = (len(INPUT_FEATURES),)

    output_shape = len(OUTPUT_TARGETS)

    network = AdvancedNeuralNetwork(input_shape=input_shape, output_shape=output_shape, config=config_payload)

    network.compile_model()

    return network.model

def load_model_checkpoint(paths: Dict[str, Path], model_config: Dict[str, Any], training_config: Dict[str, Any], checkpoint_index: pd.DataFrame, checkpoint_name: Optional[str] = None) -> Tuple[Optional[tf.keras.Model], Optional[Dict[str, Any]]]:
    """Load model weights from the selected checkpoint."""
    if checkpoint_index.empty: return None, None

    selected_row = checkpoint_index.iloc[-1] if checkpoint_name is None else checkpoint_index.loc[checkpoint_index["name"] == checkpoint_name].iloc[0]

    weights_path = Path(selected_row["path"])

    tf.keras.backend.clear_session()

    model = build_model_from_config(model_config=model_config, training_config=training_config)

    model.load_weights(weights_path)

    metadata = {
        "epoch": int(selected_row["epoch"]),
        "weights_path": str(weights_path),
        "size_bytes": int(selected_row["size_bytes"]),
        "modified": selected_row["modified"]
    }

    return model, metadata

def compute_predictions(model: Optional[tf.keras.Model], scaler_X: Any, scaler_y: Any, particle_df: pd.DataFrame, sample_size: int = 256) -> Tuple[pd.DataFrame, Dict[str, float]]:
    """Generate predictions and residual analytics using stored scalers."""
    if model is None: return pd.DataFrame(), {}

    feature_subset = particle_df[INPUT_FEATURES].copy()

    if sample_size and len(feature_subset) > sample_size:
        feature_subset = feature_subset.sample(sample_size, random_state=ANALYSIS_SEED)

    scaled_inputs = scaler_X.transform(feature_subset.values) if scaler_X is not None else feature_subset.values

    predictions_scaled = model.predict(scaled_inputs, verbose=0)

    predictions = scaler_y.inverse_transform(predictions_scaled) if scaler_y is not None else predictions_scaled

    actual_outputs = particle_df.loc[feature_subset.index, OUTPUT_TARGETS].values

    residuals = predictions - actual_outputs

    residual_df = pd.DataFrame(index=feature_subset.index)

    if "particle_id" in particle_df.columns:
        residual_df["particle_id"] = particle_df.loc[feature_subset.index, "particle_id"]

    for idx, target in enumerate(OUTPUT_TARGETS):
        residual_df[f"actual_{target}"] = actual_outputs[:, idx]

        residual_df[f"pred_{target}"] = predictions[:, idx]

        residual_df[f"residual_{target}"] = residuals[:, idx]

    residual_df["residual_norm"] = np.linalg.norm(residuals, axis=1)

    mae_value = float(np.mean(np.abs(residuals)))

    rmse_value = float(np.sqrt(np.mean(np.square(residuals))))

    metrics = {
        "samples": int(len(residual_df)),
        "mae": mae_value,
        "rmse": rmse_value
    }

    return residual_df, metrics

def summarize_run_performance(results_df: pd.DataFrame, epoch_summary: pd.DataFrame) -> pd.DataFrame:
    """Create a concise summary of key performance indicators."""
    if results_df.empty: return pd.DataFrame()

    best_epoch_idx = int(results_df["val_loss"].idxmin())

    best_row = results_df.loc[best_epoch_idx]

    final_row = results_df.iloc[-1]

    early_row = results_df.iloc[0]

    improvement = float(early_row["val_loss"] - best_row["val_loss"])

    consistency = float(epoch_summary["combined_loss_std"].tail(5).mean()) if not epoch_summary.empty else float("nan")

    summary = pd.DataFrame([
        {"metric": "Best validation loss", "value": best_row["val_loss"], "notes": f"Epoch {int(best_row['epoch'])}"},
        {"metric": "Final validation loss", "value": final_row["val_loss"], "notes": f"Train gap {final_row['train_val_gap']:.4f}"},
        {"metric": "Validation improvement", "value": improvement, "notes": "Drop from first to best epoch"},
        {"metric": "Validation stability (std last 5 epochs)", "value": consistency, "notes": "Lower is more stable"},
        {"metric": "Average epoch time (last 10 epochs)", "value": results_df["epoch_time"].tail(10).mean(), "notes": "Supports batch-size experiments"}
    ])

    return summary

def suggest_hyperparameters(model_config: Dict[str, Any], training_config: Dict[str, Any], config_history: pd.DataFrame, results_df: pd.DataFrame) -> pd.DataFrame:
    """Derive hyperparameter sweep recommendations from observed metrics."""
    if results_df.empty: return pd.DataFrame()

    suggestions: List[Dict[str, Any]] = []

    base_lr = float(model_config.get("learning_rate", 0.001))

    final_window = results_df.tail(5)

    val_loss_range = float(final_window["val_loss"].max() - final_window["val_loss"].min())

    best_epoch = int(results_df.loc[results_df["val_loss"].idxmin(), "epoch"])

    final_epoch = int(results_df.iloc[-1]["epoch"])

    if val_loss_range < 0.01 and final_epoch - best_epoch > 5:
        proposals = [round(base_lr * factor, 6) for factor in (0.5, 0.8, 1.2)]

        suggestions.append({
            "parameter": "learning_rate",
            "proposed_values": proposals,
            "rationale": "Validation loss plateaued across the last epochs; nudging the optimizer step can reintroduce progress.",
            "constraints": "Keep BinaryWeightConstraintMax(max_binary_digits=5) to preserve numerical stability."
        })

    train_val_gap = float(final_window["val_loss"].mean() - final_window["train_loss"].mean())

    if train_val_gap > 0.05:
        suggestions.append({
            "parameter": "dropout_rate",
            "proposed_values": [0.1, 0.15, 0.2],
            "rationale": "Consistent validation > training loss points to mild overfitting; light dropout can regularize activations.",
            "constraints": "Ensure enable_weight_oscillation_dampener remains True to counter oscillatory weight updates."
        })

    avg_epoch_time = float(results_df["epoch_time"].tail(10).mean())

    if avg_epoch_time < 1.5:
        baseline_batch = int(training_config.get("batch_size", 16))

        candidate_batches = sorted({baseline_batch, 24, 32})

        suggestions.append({
            "parameter": "batch_size",
            "proposed_values": candidate_batches,
            "rationale": "Headroom in epoch time suggests larger batches could reduce gradient noise without memory pressure.",
            "constraints": "Verify GPU memory against recorded peak 361 MB before scaling further."
        })

    if suggestions:
        recommendations = pd.DataFrame(suggestions)

        return recommendations

    return pd.DataFrame()

In [11]:
paths = resolve_project_paths()

display(Markdown(f"**Project root:** `{paths['project_root']}`"))

artifact_status = validate_required_artifacts(paths)

display(artifact_status)

**Project root:** `c:\Users\jesse\OneDrive\Documents\Programming Projects\Neural Network Lab - Python\neural-network-lab-python`

,artifact,path,exists,size_bytes
0,model_config,c:\Users\jesse\OneDrive\Documents\Programming ...,True,186
1,training_config,c:\Users\jesse\OneDrive\Documents\Programming ...,True,36
2,loss_history,c:\Users\jesse\OneDrive\Documents\Programming ...,True,170257
3,training_results,c:\Users\jesse\OneDrive\Documents\Programming ...,True,11952
4,configuration_log,c:\Users\jesse\OneDrive\Documents\Programming ...,True,604
5,particle_data,c:\Users\jesse\OneDrive\Documents\Programming ...,True,2747
6,scaler_X,c:\Users\jesse\OneDrive\Documents\Programming ...,True,807
7,scaler_y,c:\Users\jesse\OneDrive\Documents\Programming ...,True,759


In [12]:
model_config, training_config, config_history = load_configs(paths)

display(Markdown("### Active Model Configuration"))

display(pd.Series(model_config, name="model_config"))

display(Markdown("### Active Training Configuration"))

display(pd.Series(training_config, name="training_config"))

if not config_history.empty:
    display(Markdown("### Historical Configuration Snapshots"))

    display(config_history.sort_values("timestamp"))

### Active Model Configuration

hidden_layers                         [64, 32, 16]
activation                                    relu
optimizer                                     adam
learning_rate                               0.0050
dropout_rate                                0.0000
enable_weight_oscillation_dampener            True
Name: model_config, dtype: object

### Active Training Configuration

epochs        60
batch_size    16
Name: training_config, dtype: int64

### Historical Configuration Snapshots

,config_id,timestamp,hidden_layers,activation,optimizer,learning_rate,dropout_rate,enable_weight_oscillation_dampener,train_epochs,train_batch_size,best_r2,final_r2,best_epoch,avg_epoch_time,total_training_time,weight_modifications_used
0,training_config_20250929_235603,2025-09-29 23:56:03.826720,"[64, 32, 16]",relu,adam,0.0050,0.0000,True,60,16,0.8539,0.8432,35,1.0948,65.8267,[oscillation_dampening]
1,training_config_20250929_235859,2025-09-29 23:58:59.524273,"[64, 32, 16]",relu,adam,0.0050,0.0000,True,60,16,0.8372,0.8145,44,1.1421,68.7918,[oscillation_dampening]
2,training_config_20250930_000903,2025-09-30 00:09:03.426591,"[64, 32, 16]",relu,adam,0.0050,0.0000,True,60,16,0.8464,0.8326,30,1.0550,63.5310,[oscillation_dampening]


In [13]:
analytics = load_training_logs(paths)

loss_records = analytics["loss_records"]

epoch_summary = analytics["epoch_summary"]

results_df = analytics["results"]

merged_metrics = analytics["merged_metrics"]

display(Markdown("### Epoch-Level Performance Summary"))

display(results_df.tail(10))

performance_snapshot = summarize_run_performance(results_df, epoch_summary)

display(Markdown("### Key Performance Indicators"))

display(performance_snapshot)

### Epoch-Level Performance Summary

,epoch,epoch_time,memory_mb,r2_score,timestamp,train_loss,train_mae,train_mse,val_loss,val_mae,val_rmse,val_loss_delta,train_val_gap
50,50,1.0623,361.0898,0.8390,2025-09-30 00:08:53.290409,0.1231,0.1783,0.0678,0.1770,0.2755,0.4207,-0.0003,0.0539
51,51,1.0482,361.2695,0.8395,2025-09-30 00:08:54.369054,0.1249,0.1792,0.0705,0.1761,0.2726,0.4197,-0.0009,0.0512
52,52,1.0862,361.2930,0.8314,2025-09-30 00:08:55.455266,0.1272,0.1828,0.0715,0.1861,0.2909,0.4314,0.0100,0.0589
53,53,1.0535,361.3086,0.8332,2025-09-30 00:08:56.508718,0.1260,0.1810,0.0710,0.1825,0.2783,0.4272,-0.0036,0.0565
54,54,1.0346,361.3320,0.8364,2025-09-30 00:08:57.543269,0.1248,0.1806,0.0689,0.1786,0.2749,0.4227,-0.0039,0.0539
55,55,1.0591,361.3750,0.8463,2025-09-30 00:08:58.602346,0.1203,0.1743,0.0663,0.1683,0.2649,0.4102,-0.0104,0.0480
56,56,1.0477,361.4375,0.8441,2025-09-30 00:08:59.650084,0.1161,0.1686,0.0637,0.1720,0.2755,0.4147,0.0037,0.0559
57,57,1.0548,361.4844,0.8099,2025-09-30 00:09:00.704926,0.1152,0.1685,0.0619,0.2064,0.2827,0.4543,0.0344,0.0912
58,58,1.0541,361.4844,0.8359,2025-09-30 00:09:01.759037,0.1237,0.1780,0.0693,0.1801,0.2724,0.4244,-0.0262,0.0565
59,59,1.0483,361.4844,0.8326,2025-09-30 00:09:02.807311,0.1160,0.1703,0.0617,0.1840,0.2818,0.4289,0.0038,0.0680


### Key Performance Indicators

,metric,value,notes
0,Best validation loss,0.1683,Epoch 55
1,Final validation loss,0.1840,Train gap 0.0680
2,Validation improvement,0.3962,Drop from first to best epoch
3,Validation stability (std last 5 epochs),0.0248,Lower is more stable
4,Average epoch time (last 10 epochs),1.0549,Supports batch-size experiments


In [14]:
particle_df = load_particle_data(paths)

scaler_X, scaler_y = load_scalers(paths)

display(Markdown("### Particle Data Snapshot"))

display(particle_df.head())

display(particle_df.describe(include="all").transpose())

Loaded particle data from c:\Users\jesse\OneDrive\Documents\Programming Projects\Neural Network Lab - Python\neural-network-lab-python\particle_data.csv (10 particles)


### Particle Data Snapshot

,particle_id,mass,initial_velocity_x,initial_velocity_y,initial_position_x,initial_position_y,charge,magnetic_field_strength,simulation_time,final_velocity_x,final_velocity_y,final_position_x,final_position_y,kinetic_energy,trajectory_length
0,1,3.8079,-4.7942,1.1185,2.1509,-7.5592,1,0.4724,8.2198,-3.4993,-3.6076,-28.4755,19.5505,48.0940,40.9013
1,2,9.5121,4.6991,-3.6051,-6.5895,-0.0965,0,0.1859,1.6710,4.6573,-3.6790,1.3369,-6.3952,167.5366,10.1244
2,3,7.3467,3.3244,-2.0786,-8.6990,-9.3122,0,0.7181,9.8820,3.4632,-2.0477,22.8996,-29.9656,59.4604,37.7496
3,4,6.0267,-2.8766,-1.3364,8.9777,8.1864,0,0.8385,7.9502,-2.7367,-1.2832,-12.8532,-2.3969,27.5301,24.2610
4,5,1.6446,-3.1818,-0.4393,9.3126,-4.8244,0,0.6156,2.7884,-2.6453,-0.4595,0.4317,-6.0678,5.9275,8.9675


,count,mean,std,min,25%,50%,75%,max
particle_id,10.0000,5.5000,3.0277,1.0000,3.2500,5.5000,7.7500,10.0000
mass,10.0000,5.2494,3.1271,0.6750,2.1854,6.0389,7.2875,9.5121
initial_velocity_x,10.0000,-1.0473,3.0237,-4.7942,-3.0936,-2.0226,0.0155,4.6991
initial_velocity_y,10.0000,-0.9961,2.3386,-4.5355,-2.7721,-0.8878,0.7287,2.8518
initial_position_x,10.0000,0.1854,6.8619,-8.6990,-5.9191,0.4770,5.5471,9.3126
initial_position_y,10.0000,-1.9089,5.4081,-9.3122,-5.9333,-1.9311,0.8010,8.1864
charge,10.0000,0.3000,0.4830,0.0000,0.0000,0.0000,0.7500,1.0000
magnetic_field_strength,10.0000,0.7416,0.4193,0.1859,0.5082,0.6760,0.8233,1.6746
simulation_time,10.0000,6.2764,3.1645,1.0497,3.9318,7.7512,8.1524,9.8820
final_velocity_x,10.0000,-0.9149,2.8501,-3.4993,-2.7138,-2.0649,-0.1479,4.6573


In [15]:
checkpoint_index = list_checkpoint_weights(paths)

display(Markdown("### Available Weight Checkpoints"))

display(checkpoint_index)

model, checkpoint_meta = load_model_checkpoint(paths, model_config, training_config, checkpoint_index)

if checkpoint_meta is not None:
    display(Markdown(f"Loaded checkpoint: **epoch {checkpoint_meta['epoch']}** from `{checkpoint_meta['weights_path']}`"))

    display(pd.Series(checkpoint_meta))

### Available Weight Checkpoints

,epoch,name,path,modified,size_bytes
0,0,model_weights_epoch_0.weights.h5,c:\Users\jesse\OneDrive\Documents\Programming ...,2025-09-30 05:08:00.758926153,68880
1,10,model_weights_epoch_10.weights.h5,c:\Users\jesse\OneDrive\Documents\Programming ...,2025-09-30 05:08:11.309141397,68880
2,20,model_weights_epoch_20.weights.h5,c:\Users\jesse\OneDrive\Documents\Programming ...,2025-09-30 05:08:21.791495562,68880
3,30,model_weights_epoch_30.weights.h5,c:\Users\jesse\OneDrive\Documents\Programming ...,2025-09-30 05:08:32.316653490,68880
4,40,model_weights_epoch_40.weights.h5,c:\Users\jesse\OneDrive\Documents\Programming ...,2025-09-30 05:08:42.764005184,68880
5,50,model_weights_epoch_50.weights.h5,c:\Users\jesse\OneDrive\Documents\Programming ...,2025-09-30 05:08:53.320873976,68880
6,59,model_weights_epoch_59.weights.h5,c:\Users\jesse\OneDrive\Documents\Programming ...,2025-09-30 05:09:02.842465639,68880


c:\Users\jesse\OneDrive\Documents\Programming Projects\Neural Network Lab - Python\neural-network-lab-python\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Loaded checkpoint: **epoch 59** from `c:\Users\jesse\OneDrive\Documents\Programming Projects\Neural Network Lab - Python\neural-network-lab-python\model_weights_epoch_59.weights.h5`

epoch                                                          59
weights_path    c:\Users\jesse\OneDrive\Documents\Programming ...
size_bytes                                                  68880
modified                            2025-09-30 05:09:02.842465639
dtype: object

In [16]:
residuals_df, residual_metrics = compute_predictions(model, scaler_X, scaler_y, particle_df)

if residual_metrics:
    display(Markdown("### Residual Metrics"))

    display(pd.Series(residual_metrics, name="residual_metrics"))

if not residuals_df.empty:
    display(Markdown("### Residual Sample"))

    display(residuals_df.head())

### Residual Metrics

samples   10.0000
mae        2.9416
rmse       4.9856
Name: residual_metrics, dtype: float64

### Residual Sample

,particle_id,actual_final_velocity_x,pred_final_velocity_x,residual_final_velocity_x,actual_final_velocity_y,pred_final_velocity_y,residual_final_velocity_y,actual_final_position_x,pred_final_position_x,residual_final_position_x,actual_final_position_y,pred_final_position_y,residual_final_position_y,actual_kinetic_energy,pred_kinetic_energy,residual_kinetic_energy,actual_trajectory_length,pred_trajectory_length,residual_trajectory_length,residual_norm
0,1,-3.4993,-1.7679,1.7314,-3.6076,-3.3072,0.3004,-28.4755,-25.9221,2.5535,19.5505,25.0822,5.5317,48.0940,41.3907,-6.7034,40.9013,43.5427,2.6414,9.5979
1,2,4.6573,4.6524,-0.0050,-3.6790,-2.6700,1.0090,1.3369,4.6395,3.3026,-6.3952,-4.9603,1.4349,167.5366,139.4686,-28.0680,10.1244,9.0320,-1.0923,28.3371
2,3,3.4632,4.2090,0.7458,-2.0477,-2.8503,-0.8026,22.8996,28.1209,5.2212,-29.9656,-34.3307,-4.3651,59.4604,65.4358,5.9755,37.7496,40.4263,2.6767,9.5072
3,4,-2.7367,-2.3808,0.3558,-1.2832,-0.8082,0.4750,-12.8532,-16.2292,-3.3760,-2.3969,0.1324,2.5292,27.5301,29.0890,1.5589,24.2610,26.2540,1.9930,4.9547
4,5,-2.6453,-2.3451,0.3002,-0.4595,-0.2851,0.1744,0.4317,-0.4138,-0.8455,-6.0678,-6.2952,-0.2275,5.9275,9.0566,3.1290,8.9675,7.0576,-1.9099,3.7849


In [17]:
figures_dir = paths["figures_dir"]

# Loss trend
fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data=results_df, x="epoch", y="train_loss", ax=ax, label="Train Loss")
sns.lineplot(data=results_df, x="epoch", y="val_loss", ax=ax, label="Validation Loss")
ax.fill_between(results_df["epoch"], results_df["val_loss"] - results_df["val_loss"].rolling(5, min_periods=1).std(), results_df["val_loss"] + results_df["val_loss"].rolling(5, min_periods=1).std(), color="tab:blue", alpha=0.1)
ax.set_title("Training vs Validation Loss")
ax.set_ylabel("Loss")
fig.tight_layout()
loss_curve_path = figures_dir / "loss_curves.png"
fig.savefig(loss_curve_path, dpi=200)
plt.close(fig)
display(Markdown(f"Saved loss curves to `{loss_curve_path}`"))

# Learning rate vs final loss
if not config_history.empty:
    lr_df = config_history.copy()

    fig, ax = plt.subplots(figsize=(8, 5))
    sns.scatterplot(data=lr_df, x="learning_rate", y="final_r2", size="total_training_time", hue="final_r2", palette="viridis", ax=ax)
    ax.set_title("Learning Rate vs Final R²")
    ax.set_xlabel("Learning Rate")
    ax.set_ylabel("Final R²")
    fig.tight_layout()
    lr_plot_path = figures_dir / "learning_rate_vs_r2.png"
    fig.savefig(lr_plot_path, dpi=200)
    plt.close(fig)
    display(Markdown(f"Saved learning-rate diagnostics to `{lr_plot_path}`"))

# Residual histogram
if not residuals_df.empty:
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.histplot(residuals_df["residual_norm"], bins=30, ax=ax, kde=True, color="tab:orange")
    ax.set_title("Residual Norm Distribution")
    ax.set_xlabel("Residual Norm")
    fig.tight_layout()
    residual_hist_path = figures_dir / "residual_norm_hist.png"
    fig.savefig(residual_hist_path, dpi=200)
    plt.close(fig)
    display(Markdown(f"Saved residual histogram to `{residual_hist_path}`"))

# Correlation heatmap
heatmap_features = ["train_loss", "val_loss", "train_mae", "val_mae", "r2_score", "epoch_time", "train_val_gap"]
usable_cols = [col for col in heatmap_features if col in merged_metrics.columns]

if usable_cols:
    corr_matrix = merged_metrics[usable_cols].corr()

    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", ax=ax)
    ax.set_title("Metric Correlation Heatmap")
    fig.tight_layout()
    heatmap_path = figures_dir / "metric_correlation_heatmap.png"
    fig.savefig(heatmap_path, dpi=200)
    plt.close(fig)
    display(Markdown(f"Saved correlation heatmap to `{heatmap_path}`"))

Saved loss curves to `c:\Users\jesse\OneDrive\Documents\Programming Projects\Neural Network Lab - Python\neural-network-lab-python\training_output\analysis\figures\loss_curves.png`

Saved learning-rate diagnostics to `c:\Users\jesse\OneDrive\Documents\Programming Projects\Neural Network Lab - Python\neural-network-lab-python\training_output\analysis\figures\learning_rate_vs_r2.png`

Saved residual histogram to `c:\Users\jesse\OneDrive\Documents\Programming Projects\Neural Network Lab - Python\neural-network-lab-python\training_output\analysis\figures\residual_norm_hist.png`

Saved correlation heatmap to `c:\Users\jesse\OneDrive\Documents\Programming Projects\Neural Network Lab - Python\neural-network-lab-python\training_output\analysis\figures\metric_correlation_heatmap.png`

In [18]:
recommendations_df = suggest_hyperparameters(model_config, training_config, config_history, results_df)

if not recommendations_df.empty:
    display(Markdown("### Recommended Hyperparameter Sweeps"))

    display(recommendations_df)

else:
    display(Markdown("No immediate hyperparameter adjustments detected beyond current configuration."))

### Recommended Hyperparameter Sweeps

,parameter,proposed_values,rationale,constraints
0,dropout_rate,"[0.1, 0.15, 0.2]",Consistent validation > training loss points t...,Ensure enable_weight_oscillation_dampener rema...
1,batch_size,"[16, 24, 32]",Headroom in epoch time suggests larger batches...,Verify GPU memory against recorded peak 361 MB...


In [19]:
insight_items: List[str] = []

if not results_df.empty:
    final_row = results_df.iloc[-1]

    best_row = results_df.loc[results_df["val_loss"].idxmin()]

    insight_items.append(f"Best validation loss {best_row['val_loss']:.4f} at epoch {int(best_row['epoch'])}.")

    insight_items.append(f"Validation plateau range over last window: {(results_df.tail(5)['val_loss'].max() - results_df.tail(5)['val_loss'].min()):.4f}.")

    insight_items.append(f"Train/val gap at final epoch: {final_row['train_val_gap']:.4f}.")

if residual_metrics:
    insight_items.append(f"Mean absolute residual across sampled predictions: {residual_metrics['mae']:.4f}.")

if not insight_items:
    insight_items.append("Insufficient data to derive insights.")

display(Markdown("### Insight Summary"))

for item in insight_items:
    display(Markdown(f"- {item}"))

### Insight Summary

- Best validation loss 0.1683 at epoch 55.

- Validation plateau range over last window: 0.0381.

- Train/val gap at final epoch: 0.0680.

- Mean absolute residual across sampled predictions: 2.9416.

In [20]:
def run_notebook_smoke_test() -> Dict[str, Any]:
    """Validate that core notebook stages complete without exceptions."""
    status = {
        "artifacts_present": artifact_status["exists"].all(),
        "config_history_entries": int(len(config_history)),
        "loss_records": int(len(loss_records)),
        "results_records": int(len(results_df)),
        "residual_samples": int(len(residuals_df)),
        "recommendations": int(len(recommendations_df))
    }

    return status

smoke_test_status = run_notebook_smoke_test()

display(Markdown("### Validation Checklist"))

display(pd.Series(smoke_test_status, name="notebook_validation"))

### Validation Checklist

artifacts_present         True
config_history_entries       3
loss_records              2400
results_records             60
residual_samples            10
recommendations              2
Name: notebook_validation, dtype: object

## Actionable Next Steps

- Re-run the training pipeline after trialing the proposed learning-rate, dropout, and batch-size combinations; capture new config snapshots for comparison.
- Promote saved figures under `training_output/analysis/figures/` into experiment reports or dashboards.
- Extend this notebook with automated sweeps (GridSearch or Bayesian optimization) once additional configuration diversity is available.

### Reuse Tips

- Parameterize `sample_size` within `compute_predictions` to scale residual analysis for larger datasets.
- Import this notebook’s helper functions via `%run experiment_analysis_framework.ipynb` inside future analysis notebooks for rapid setup.
- Store additional diagnostics (e.g., feature importance, SHAP values) within the `analysis` directory for cross-experiment benchmarking.